In [34]:
import QuantLib as ql
import pandas as pd
import numpy as np

In [35]:
tasa_cupon = 0.0713
periodicidad = 2

In [36]:
precio = 102.42

In [37]:
fecha_emision = pd.to_datetime('2015-12-22')
fecha_vence = pd.to_datetime('2021-12-22')
fecha_calculo = pd.to_datetime('2020-02-12')

In [38]:
fecha_calc_ql = ql.Date(fecha_calculo.day,fecha_calculo.month,fecha_calculo.year)

ql.Settings.instance().evaluationDate = fecha_calc_ql

In [39]:
qlCalendar = ql.BespokeCalendar('Costa Rica')
qlCalendar.addWeekend(ql.Sunday)
qlCalendar.addWeekend(ql.Saturday)
qlCalendar.addHoliday(ql.Date(25,12,2020))
qlCalendar.addHoliday(ql.Date(1,1,2021))
qlCalendar.addHoliday(ql.Date(15,9,2020))

In [40]:
qlCalendar.holidayList(ql.Date(1,1,2020),ql.Date(1,1,2022),False)

(Date(15,9,2020), Date(25,12,2020), Date(1,1,2021))

In [41]:
intervalo = ql.Period(60,ql.Days)
print(fecha_calc_ql)

February 12th, 2020


In [42]:
fecha = fecha_calc_ql + intervalo
print(fecha)

April 12th, 2020


In [43]:
fecha_1 = qlCalendar.advance(fecha_calc_ql,intervalo)
print(fecha_1)

May 6th, 2020


In [44]:
fecha_vence_ql = ql.Date(fecha_vence.day,fecha_vence.month,fecha_vence.year)
fecha_emision_ql = ql.Date(fecha_emision.day,fecha_emision.month,fecha_emision.year)

In [45]:
qlTenor = ql.Period(periodicidad)
qlConvencion = ql.Following
dateGeneration = ql.DateGeneration.Backward
monthEnd = False

In [46]:
cronograma = ql.Schedule(fecha_emision_ql,fecha_vence_ql,qlTenor,
                         qlCalendar,qlConvencion,qlConvencion,dateGeneration,monthEnd)
# list(cronograma)
pd.DataFrame({'date':list(cronograma)})

,date
0,"December 22nd, 2015"
1,"June 22nd, 2016"
2,"December 22nd, 2016"
3,"June 22nd, 2017"
4,"December 22nd, 2017"
5,"June 22nd, 2018"
6,"December 24th, 2018"
7,"June 24th, 2019"
8,"December 23rd, 2019"
9,"June 22nd, 2020"


In [47]:
dayCount = ql.Thirty360(ql.Thirty360.BondBasis)

In [48]:
compound_type = ql.Compounded

In [49]:
frequency = ql.Semiannual

In [50]:
settlementDays = 0

In [51]:
interest_rate = ql.InterestRate(tasa_cupon,dayCount,compound_type,frequency)
print(interest_rate)

7.130000 % 30/360 (Bond Basis) Semiannual compounding


In [52]:
print(interest_rate.compoundFactor(1))

1.0725709225


In [53]:
print(interest_rate.discountFactor(4))

0.7556069152825599


In [54]:
nueva_tasa = interest_rate.equivalentRate(compound_type,ql.Annual,1)
print(nueva_tasa)

7.257092 % 30/360 (Bond Basis) Annual compounding


### Creación del bono

In [55]:
bono = ql.FixedRateBond(settlementDays,100,cronograma,[tasa_cupon],dayCount)
bono

<QuantLib.QuantLib.FixedRateBond; proxy of <Swig Object of type 'ext::shared_ptr< FixedRateBond > *' at 0x0000020F2006ABE0> >

In [56]:
cf = bono.cashflows()
fechas = [item.date() for item in cf]
montos = [item.amount() for item in cf]
print(pd.DataFrame({'Fechas':fechas,'Montos':montos}))

                 Fechas      Montos
0       June 22nd, 2016    3.565000
1   December 22nd, 2016    3.565000
2       June 22nd, 2017    3.565000
3   December 22nd, 2017    3.565000
4       June 22nd, 2018    3.565000
5   December 24th, 2018    3.604611
6       June 24th, 2019    3.565000
7   December 23rd, 2019    3.545194
8       June 22nd, 2020    3.545194
9   December 22nd, 2020    3.565000
10      June 22nd, 2021    3.565000
11  December 22nd, 2021    3.565000
12  December 22nd, 2021  100.000000


In [57]:
rend = bono.bondYield(precio,dayCount,compound_type,frequency)

In [58]:
rend

0.057349736690521236

In [59]:
interes_rend = ql.InterestRate(rend,dayCount,compound_type,frequency)

In [60]:
duracion=ql.BondFunctions.duration(bono,interes_rend,ql.Duration.Macaulay)
duracion

1.7619109199149028

In [61]:
duracionMod=ql.BondFunctions.duration(bono,interes_rend,ql.Duration.Modified)
duracionMod

1.7127967000390851

In [62]:
convexidad=ql.BondFunctions.convexity(bono,interes_rend)
convexidad

3.8669145300256282

#### Curvas de rendimiento

In [64]:
curvas = pd.read_csv("curvas(1).csv")

FileNotFoundError: [Errno 2] No such file or directory: 'curvas(1).csv'

In [ ]:
curvas

In [ ]:
curvas.dtypes

In [ ]:
curvas['DATE'] = pd.to_datetime(curvas['DATE'])

In [ ]:
tabla_curvas = curvas.pivot(values = 'RATE',index = 'DATE', columns = 'TERM')
tabla_curvas.head()

In [ ]:
tabla_curvas = tabla_curvas.loc[:,0:900]

In [ ]:
tabla_curvas.head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(tabla_curvas[360])

In [ ]:
tabla_curvas = tabla_curvas.loc[tabla_curvas.index >= pd.to_datetime('2018-04-01'),:]
plt.plot(tabla_curvas[360])

In [ ]:
plt.plot(tabla_curvas.iloc[200,:])

In [ ]:
curva_actual = tabla_curvas.iloc[-1,:]
plt.plot(curva_actual)

In [ ]:
tasa_actual = curva_actual.values
plazos = curva_actual.index.values
plazos

In [ ]:
curva_ql0 = ql.ZeroCurve(fecha_calc_ql+plazos,tasa_actual,ql.Thirty360(ql.Thirty360.BondBasis))
curva_ql = ql.YieldTermStructureHandle(curva_ql0)

In [ ]:
estructura_plazos = ql.ZeroSpreadedTermStructure(
        curva_ql,ql.QuoteHandle(ql.SimpleQuote(0)), ql.Compounded,
        ql.Semiannual,dayCount)

In [ ]:
pricing_engine = ql.DiscountingBondEngine(
        ql.RelinkableYieldTermStructureHandle(estructura_plazos))

In [ ]:
bono.setPricingEngine(pricing_engine)
bono.NPV()

In [ ]:
def Precio(spread):
    estructura_plazos = ql.ZeroSpreadedTermStructure(
            curva_ql,ql.QuoteHandle(ql.SimpleQuote(spread)), ql.Compounded,
            ql.Semiannual,dayCount)
    pricing_engine = ql.DiscountingBondEngine(
            ql.RelinkableYieldTermStructureHandle(estructura_plazos))
    
    bono.setPricingEngine(pricing_engine)
    return bono.NPV()

In [ ]:
Precio(-0.01)

In [ ]:
def ErrorPrecio(spread):
    return precio-Precio(spread)

In [ ]:
ErrorPrecio(0)

In [ ]:
from scipy.optimize import bisect

In [ ]:
spread_estimado = bisect(ErrorPrecio,-1,1)
spread_estimado

In [ ]:
Precio(spread_estimado)

## Cálculo de valor en riesgo por método paramétrico

Aproximación por variaciones paralelas

$VaR = \sigma \times \Phi^{-1}(q) \times W = D P \,\sigma_r \,z_q  W/100$

In [ ]:
duracionMod

In [ ]:
from scipy.stats import norm

In [ ]:
nivel_conf = 0.999
zq = norm.ppf(nivel_conf)
zq

In [ ]:
cambios_tasas = tabla_curvas.diff(20).iloc[20:]
cambios_tasas.head()

In [ ]:
sigma_plazo=cambios_tasas.std()
plt.plot(sigma_plazo)

In [ ]:
vol=sigma_plazo[600]
vol

In [ ]:
VaR = precio*duracionMod*zq*vol/100
VaR

In [ ]:
duracionMod

In [ ]:
delta = .0001
nuevo_precio = Precio(spread_estimado+delta)
cambio_porcentual = (1/precio)*(precio-nuevo_precio)/(delta)

In [ ]:
from sklearn import decomposition

In [ ]:
pca = decomposition.PCA(n_components=3, whiten = False)

In [ ]:
cambios_tasas = np.diff(tabla_curvas.values,axis = 0)

In [ ]:
x_r = pca.fit_transform(cambios_tasas)
x_r.shape

In [ ]:
# Eje Y maginitud de la componente principal. Se puede indentificar que los datos de corto plazo están represnetados en 
# esta primera componente principal (los datos que tienen más varianza)

comp = pca.components_
plt.plot(plazos,comp[0])



In [ ]:
# Eje Y maginitud de la componente principal. Se puede indentificar que los datos de mediano plazo están represnetados en 
# esta segunda componente principal (los datos que tienen más varianza)

comp = pca.components_
plt.plot(plazos,comp[1])

In [ ]:
# Eje Y maginitud de la componente principal. Se puede indentificar que los datos de largo plazo están represnetados en 
# esta tercera componente principal (los datos que tienen más varianza)

comp = pca.components_
plt.plot(plazos,comp[2])

In [ ]:
# Autovalores son las varianzas de cada componente

print(pca.explained_variance_ratio_)

In [ ]:
# Componentes que explican la varianza

print(np.cumsum(pca.explained_variance_ratio_))

In [ ]:
def PriceTermStructShift(spread,shift):
    ql_curvacero_shift = ql.ZeroCurve(fecha_calc_ql+plazos,tasa_actual+shift,ql.Thirty360(ql.Thirty360.BondBasis))
    ql_curva_shift = ql.YieldTermStructureHandle(ql_curvacero_shift)    
    term_structure_spread = ql.ZeroSpreadedTermStructure(ql_curva_shift,ql.QuoteHandle(ql.SimpleQuote(spread)),ql.Compounded,ql.Semiannual,dayCount) 
    pricing_engine_shift = ql.DiscountingBondEngine(ql.RelinkableYieldTermStructureHandle(term_structure_spread))
    bono.setPricingEngine(pricing_engine_shift)
    return bono.NPV()

In [ ]:
PriceTermStructShift(spread_estimado, .02)

In [ ]:
dur_pca = np.zeros(3)
delta = .0001


# Sensibilidad: cuanto cambia el precio respecto a la varianza
for icomp in range(0,3):
    nuevo_precio = PriceTermStructShift(spread_estimado,delta*comp[icomp])
    dur_pca[icomp] = -(1/precio)*(precio-nuevo_precio)/(delta)
print(dur_pca)


# Es más fuerte al final porque es donde recibo casi toda la plata, entonces tengo más cúmulo de riesgo pero cuya variabilidad es menor. 

In [ ]:
np.linalg.norm(comp[0])

In [ ]:
covar = np.cov(np.transpose(x_r))
covar

In [ ]:
sigma = np.sqrt(np.dot(np.dot(np.transpose(dur_pca),covar),dur_pca))
print(sigma)

In [ ]:
VaR = precio*sigma*zq/100
print(VaR)

Valor en riesgo por simulación histórica

In [ ]:
dur_pca = np.zeros(3)
delta = .0001


# Sensibilidad: cuanto cambia el precio respecto a la varianza
for icomp in range(0,3):
    nuevo_precio = PriceTermStructShift(spread_estimado,cambios_tasas[icomp,])
    dur_pca[icomp] = -(1/precio)*(precio-nuevo_precio)/(delta)
print(dur_pca)



In [ ]:
np.linalg.norm(comp[0])

In [ ]:
covar = np.cov(np.transpose(x_r))
covar

In [ ]:
sigma = np.sqrt(np.dot(np.dot(np.transpose(dur_pca),covar),dur_pca))
print(sigma)

In [ ]:
n=len(cambios_tasas)
precios_esc = np.zeros(n)
for i in range(0,n):
    precios_esc[i] = PriceTermStructShift(spread_estimado,cambios_tasas[i,:])

In [ ]:
import scipy as sp
from statsmodels.distributions.empirical_distribution import ECDF

In [ ]:
precio_ecdf = ECDF(precios_esc)
mu = np.mean(precios_esc)
sig = np.std(precios_esc)

In [ ]:
norm_approx = sp.stats.norm.cdf(precio_ecdf.x,mu,sig)
plt.plot(precio_ecdf.x,precio_ecdf.y,precio_ecdf.x,norm_approx)